In [ ]:
import json
import re
from pprint import pprint

import numpy as np
import pandas as pd
import dask.dataframe as dd
import pymorphy2
from IPython.core.display import HTML, display
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.dummy import DummyClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline


import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import classification_report
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import learning_curve
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import time

# Загрузка и предобработка данных

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Byte_data_hack/train.csv", index_col = "index")
print(f"Размер: {train.shape}\n")
print("Типы столбцов: ")
display(train.dtypes)
print("Фрагмент данных: ")
display(train.head())

Размер: (30000, 3)

Типы столбцов: 


name           object
description    object
target          int64
dtype: object

Фрагмент данных: 


,name,description,target
index,,,
324865089,продавец кассир,<strong>Обязанности:</strong> <ul> <li>работа ...,-1
169467135,продавец мила (шевченко 17),<p><strong>Магазин МИЛА по адресу б-р Шевченко...,-1
169939030,кассир в пиццерию г витебск,"<p><strong>Устал искать работу? Может, хочешь ...",5223
31956044,начальник строительного участка,<p>Компании &quot;Артель-Строй&quot; на постоя...,-1
36781653,продавец кассир (санкт петербург бухарестская 31),<p>Обязанности:</p><p> •Обслуживание покупа...,-1


Вам будут предоставлены 30 тыс. вакансий для обучения модели - но только для 15 тыс. из них снабжены метками класса. Как использовать оставшиеся 15 тыс. примеров - решает каждая команда индивидуально. Для демонстрации просто выбросим их.

In [ ]:
train = train.query("target != -1")
train_to_class = train.query("target == -1")
print(f"{train.shape}")
display(train.head())

(15650, 3)


,name,description,target
index,,,
169939030,кассир в пиццерию г витебск,"<p><strong>Устал искать работу? Может, хочешь ...",5223
169293782,продавец консультант yota (тц галерея),<p>За любыми достижениями нашей компании в пер...,5223
291073919,электросварщик накс ск (п 1 г норильск),<p>Группе компаний КСК на промышленный объект ...,7212
179260831,кассир,Правила приема и проведения расчетных и конвер...,5223
39608878,оператор машинного доения,"Выполнять все виды работ, связанные с производ...",6121


Описания вакансий представлены в виде HTML-верстки.

In [ ]:
display(HTML(train.at[324865089, "description"]))

In [ ]:
train.at[169939030, "description"]

Для вашего удобства произведена первичная преобработка текстового описания, из которого были выделены логические блоки: "обязанности", "требования" и т.д. Предобработанные описания хранятся в формате `JSON`. Загрузим один из файлов и посмотрим, как выглядят эти данные.

In [ ]:
df_json = pd.read_json('/content/drive/MyDrive/Colab Notebooks/Byte_data_hack/all_parsed.json')
df_json['Content'] = df_json['Content'].map(lambda p: " ".join(p[list(p.keys())[0]][:min(len(p[list(p.keys())[0]]), 2)]))
df_json.head()

,ID,Position,Content
0,92817316,Полицейский,Заработная плата от 57.000 рублей (+ премии по...
1,92823254,Мойщик автомобилей,Производить мойку автотранспортных средств на ...
2,92827184,"Заместитель директора магазина (Солнечногорск,...",Организация работы магазина Контроль работы со...
3,92830509,Офис-менеджер,Встреча клиентов компании Контроль соблюдения ...
4,92833853,Разнорабочий,фасовка комплектовка


In [ ]:
dask_df = dd.from_pandas(df_json, npartitions = 10)
dask_df = dask_df.set_index('ID')

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import nltk
from nltk.corpus import stopwords
  
def modification(p):
  p = p.lower()
  p = re.sub(patt, " ", p)
  p_new = ''
  for i in p.split():
    if i not in stop_words and len(i) > 2:
      p_new += ' ' + str(i)#lemmatizator.parse(str(i))[0].normal_form
  return p_new


patt = re.compile("[^\w\s]")
stop_words = stopwords.words('russian')
lemmatizator = pymorphy2.MorphAnalyzer()

dask_df['descr'] = dask_df['Position'] + " " + dask_df['Content']
dask_df['descr'] = dask_df['descr'].fillna("").map(modification)

In [ ]:
dask_df = dask_df['descr']

In [ ]:
responsibilities = dask_df.compute()

In [ ]:
train["full_description"] = responsibilities
train["full_description"] = train["full_description"].fillna('')
display(train.head(3))
display(train.info())

,name,description,target,full_description
index,,,,
169939030,кассир в пиццерию г витебск,"<p><strong>Устал искать работу? Может, хочешь ...",5223,кассир пиццерию витебск тебе почасовая оплата...
169293782,продавец консультант yota (тц галерея),<p>За любыми достижениями нашей компании в пер...,5223,продавец консультант yota галерея любыми дост...
291073919,электросварщик накс ск (п 1 г норильск),<p>Группе компаний КСК на промышленный объект ...,7212,электросварщик накс норильск ручная дуговая с...


<class 'pandas.core.frame.DataFrame'>
Int64Index: 15650 entries, 169939030 to 39324585
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   name              15650 non-null  object
 1   description       15645 non-null  object
 2   target            15650 non-null  int64 
 3   full_description  15650 non-null  object
dtypes: int64(1), object(3)
memory usage: 611.3+ KB


None

#Векторизация текста

In [ ]:
x_train_raw, y_train = train["full_description"], train["target"]
vectorizer = TfidfVectorizer(max_features = 1200)
x_train = vectorizer.fit_transform(x_train_raw)

In [ ]:
x_train

<15650x1200 sparse matrix of type '<class 'numpy.float64'>'
	with 133117 stored elements in Compressed Sparse Row format>

In [ ]:
x_train = x_train.toarray()

In [ ]:
x_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Byte_data_hack/test.csv", index_col = "index")
x_test.head()

,name,description
index,,
26461447,"персональный водитель сервиса ""wheely""",<p><strong>В </strong>связи с расширением авто...
26464220,менеджер по автоперевозкам,<strong>Обязанности:</strong> <ul> <li>Поиск п...
26467473,"продавец кассир (тц ""седанка сити"")",<p><strong>Твои обязанности:</strong></p> <ul>...
26468989,специалист по кадрам,"<p>Обязанности:</p> <p>- оформление приема, пе..."
26471705,администратор на телефоне,<strong>Обязанности:</strong> <ul> <li> <p>Обр...


In [ ]:
x_test["full_description"] = responsibilities
x_test["full_description"] = x_test["full_description"].fillna('')
display(x_test.head(3))
display(x_test.info())

,name,description,full_description
index,,,
26461447,"персональный водитель сервиса ""wheely""",<p><strong>В </strong>связи с расширением авто...,персональный водитель сервиса wheely выполнен...
26464220,менеджер по автоперевозкам,<strong>Обязанности:</strong> <ul> <li>Поиск п...,менеджер автоперевозкам поиск поставщиков вед...
26467473,"продавец кассир (тц ""седанка сити"")",<p><strong>Твои обязанности:</strong></p> <ul>...,продавец кассир седанка сити обслуживание пок...


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1090 entries, 26461447 to 361611956
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   name              1090 non-null   object
 1   description       1090 non-null   object
 2   full_description  1090 non-null   object
dtypes: object(3)
memory usage: 34.1+ KB


None

In [ ]:
print(x_test.info())
x_test = vectorizer.transform(x_test['full_description'])
x_test = x_test.toarray()

x_test

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1090 entries, 26461447 to 361611956
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   name              1090 non-null   object
 1   description       1090 non-null   object
 2   full_description  1090 non-null   object
dtypes: object(3)
memory usage: 34.1+ KB
None


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df_differences = pd.DataFrame(columns = ['Тип модели классификации', 'Accuracy_score', 'Precision_score', 'Recall_score', 'F1_score'])

In [ ]:
# Определяем методы

learning_models = {
    "LogisticRegression": LogisticRegression(),
    "Метод опорных векторов с линейным ядром (SVC)": svm.SVC(kernel = 'linear'),
    "Метод опорных векторов с гауссовым ядром (SVC)": svm.SVC(kernel = 'rbf'),
    "KNeighborsClassifier": KNeighborsClassifier(n_neighbors = 100),
    "Многослойный перцептрон (MLP)": MLPClassifier(max_iter = 200),
}

y_test_predict = dict()
i = 0
for name, model in learning_models.items():
    start_time = time.time()
    model.fit(x_train, y_train)
    print(f"{(time.time() - start_time)} seconds время обучения модели {name}")              
    y_test_predict[name] = model.predict(x_test)
    y_train_predict = model.predict(x_train)
    df_differences.loc[i] = [name,
                         accuracy_score(y_train, y_train_predict),
                         precision_score(y_train, y_train_predict, average = 'macro'),
                         recall_score(y_train, y_train_predict, average = 'macro'),
                         f1_score(y_train, y_train_predict, average = 'macro')]
    i += 1                

48.03229475021362 seconds время обучения модели LogisticRegression
65.24607920646667 seconds время обучения модели Метод опорных векторов с линейным ядром (SVC)
186.20962238311768 seconds время обучения модели Метод опорных векторов с гауссовым ядром (SVC)
0.020143747329711914 seconds время обучения модели KNeighborsClassifier
128.85082006454468 seconds время обучения модели Многослойный перцептрон (MLP)


In [ ]:
df_differences

,Тип модели классификации,Accuracy_score,Precision_score,Recall_score,F1_score
0,LogisticRegression,0.924856,0.555091,0.515543,0.530925
1,Метод опорных векторов с линейным ядром (SVC),0.933163,0.556890,0.530445,0.541827
2,Метод опорных векторов с гауссовым ядром (SVC),0.940383,0.562114,0.537414,0.548548
3,KNeighborsClassifier,0.436805,0.475321,0.157070,0.210212
4,Многослойный перцептрон (MLP),0.959297,0.952469,0.919411,0.932448


In [ ]:
y_test_predict["Многослойный перцептрон (MLP)"]

array([8321, 3341, 5223, ..., 5223, 8343, 4323])

In [ ]:
index_need = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Byte_data_hack/test.csv", index_col = "index").index
results = pd.DataFrame(index = index_need, columns = ['target'])
results['target'] = y_test_predict["Многослойный перцептрон (MLP)"]
results.head()

,target
index,
26461447,8321
26464220,3341
26467473,5223
26468989,3341
26471705,3343


In [ ]:
results.to_csv('/content/drive/MyDrive/Colab Notebooks/Byte_data_hack/result_3.csv')